# 18.S190/6.S090 Problem Set 4

Due Friday 3/7 at **11:59pm**; 20% penalty if it is turned in within 24 hours, and after that late psets will not be accepted.   Submit in PDF format: a decent-quality scan/image of any handwritten solutions (e.g. get a scanner app on your phone or use a tablet), combined with a PDF printout of your Jupyter notebook showing your code and (clearly labeled) results.

## Problem 1: Adaptive quadrature (10+10+10 points)

In this problem, you will compare h-adaptive quadrature with p-adaptive quadrature for the example integral:
$$
\int_0^{2\pi} \mathrm{Re}\left[\frac{1}{1 - (0.979 + 0.198i) e^{i\phi}}\right] d\phi = 2\pi
$$
(The analytical result $2\pi$ is obtained using the [residue theorem](https://en.wikipedia.org/wiki/Residue_theorem).)  If you plot this integrand, you should see that it has a very **sharp peak** near one end of the domain.

1. Implement "p-adaptive" quadrature where you integrate this function using an $N+1$-point Clenshaw–Curtis rule (using the code from pset 3), where you repeatedly double $N$, and plot the relative error vs. $N+1$ (the number of function evaluations) until the error reaches the limit of roundoff errors and stops improving.   Do the same thing for $N+1$ point trapezoidal rule, which should also converge exponentially fast since this integrand is periodic.  (Since these rules are nested, when you double $N$ you could re-use the previous function evaluations, but you don't need to implement this optimization.)

2. Implement "h-adaptive" quadrature using a fixed-order $N+1$ point Clenshaw–Curtis rule, for $N=16$:
    * Keep a list of sub-intervals $[a,b]$, starting with just a single interval $[0,2\pi]$, and estimated integral $I_N$ for each interval.
    * For each interval, also store an estimate of the error in $I_N$ by $|I_N - I_{N/2}|$, i.e. by comparing the nested $N=8$ Clenshaw–Curtis rule (which doesn't require any additional function evaluations).
    * **At each h-adaptive step:** Pick the interval with the *largest* error estimate, and break it in half into two subintervals.  Remove the interval from your list and replace it with these two subintervals, along with their estimated integrals and errors.  (A more efficient data structure for this sort of thing is a [priority queue](https://en.wikipedia.org/wiki/Priority_queue), but you don't need to worry about this.)
    * The estimated integral $I$ is just the sum of the integrals for each subinterval (and the estimated error is the sum of the estimated errors).   (In a real application, you would stop when the estimated error falls below some desired tolerance.)  Keep track of the actual relative error $|I - 2\pi|/2\pi$ after each step, as a function of the *total* number of function evaluations up to that point (including the evaluations for the intervals that were subdivided and discarded).   Plot this error vs. function evaluations (include it on the same plot as 1.1) … again, keep going until your error is close to machine precision.

3. Make a new plot with the same 3 curves as in parts 1.1 and 1.2, with the same algorithms, but for the integral $\int_0^{2\pi} 17.25 \cos(17.25\phi) d\phi = 1$, where the integrand is *not* sharply peaked.  (Clearly label all of your plots!)  What do you observe?

# Problem 2: Monte-Carlo integration (10+10+10 points)


The volume of a unit $ n $-ball (a ball of radius 1 in $ n $-dimensional space) is given by the formula:

$$
V_n = \frac{\pi^{n/2}}{\Gamma\left(\frac{n}{2} + 1\right)}
$$


where $ \Gamma $ is the gamma function. For reference, the first few values are:

- $ V_1 = 2 $ (length of a line segment from $-1$ to $1$)
- $ V_2 = \pi $ (area of a circle with radius $1$)
- $V_3 = \frac{4\pi}{3}$ (volume of a sphere with radius $1$)
- $V_4 = \frac{\pi^2}{2}$
- $V_5 = \frac{8\pi^2}{15}$

The gamma function can be computed using [`scipy.special.gamma(z)`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.gamma.html) in Python or [`SpecialFunctions.gamma(z)`](https://specialfunctions.juliamath.org/stable/functions_overview/) in Julia.

In [ ]:
# Python:
import numpy as np
from scipy.special import gamma
np.pi**(n/2) / gamma(n/2 + 1)

# Julia:
using SpecialFunctions
pi^(n/2) / gamma(n/2 + 1)

The Monte Carlo quadrature for integrating a function $ f(x) $ over a $ d $-dimensional hypercube $[-1, 1]^d$ can be computed as:

$$
\int_{[-1,1]^d} f(x) \,dx \approx \frac{2^d}{N} \sum_{i=1}^{N} f(x_i)
$$

where $ x_i $ are uniformly distributed random points in $[-1, 1]^d$ and $ 2^d $ is the volume of the hypercube.

**(1)** Write a function `monte_carlo_quadrature(f, N, d)` that returns the $N$-point Monte Carlo quadrature  of a function $f(x)$ over a $ d $-dimensional hypercube $[-1, 1]^d$, where the argument of $f$ is a $d$-component vector.   Note that the function `numpy.random.rand(d)` in Python and `rand(d)` in Julia both return a `d`-component vector uniformly distributed in $[0, 1]^d$ (not $[-1, 1]^d$); you can also generate many random points at once in a matrix with `rand(d, N)` or `rand(N, d)`, but be careful not to fill up all your memory when $N$ is large.   (To help you interpret your results, it might also be useful to compute and return the [standard deviation](https://en.wikipedia.org/wiki/Standard_deviation#Sample_standard_deviation) of the integrand at your $N$ points, which measures the spread around the mean, e.g. via `numpy.std` in Python or `Statistics.std` in Julia.)

As a check, use your function to estimate the volume of a unit $ n $-ball in dimensions $d = 1$ through $d = 3$, by integrating:

$$
f(x) :=
\begin{cases} 
   1, & \|x\|_2 < 1 \\
   0, & \|x\|_2 \ge 1
\end{cases}
$$
with $ N=100000 $ and check that the answer is roughly correct. This function can be implemented by (e.g.) `numpy.sum(x**2) < 1` in Python or `sum(x -> x^2, x) < 1` in Julia (or by using the `norm` functions in the respective linear-algebra modules).

**(2)** For each dimension $ d $ from 1 to 10, compute the relative error between your estimate and the true volume for various numbers of points $ N $, using logarithmically spaced values from $ N = 10 $ to $ N = 10^8 $.  Create a log-log plot showing the convergence rate (relative error vs. number of points) for all dimensions on the same graph.  Does it match the theoretical rate of $ O(N^{-1/2}) $ for Monte Carlo methods?

Next, we will consider the relative efficiency of Monte Carlo integration compared to deterministic quadrature methods for this problem.  Note that for piecewise smooth functions with discontinuities such as $ f(x) $ in Part 1, both the trapezoidal rule and Clenshaw–Curtis quadrature have an error of $ O(1/N_{1d}) $, where $ N_{1d} $ is the number of grid points in each dimension.  For smooth/analytic functions, from class, the trapezoidal rule converges as  $ O(1/N_{1d}^2) $ and Clenshaw–Curtis converges as  $ O(e^{-\alpha N_{1d}})$ for some $\alpha > 0$.

**(3)** For the discontinuous function $ f(x) $ in Part 1, what is the rate of convergence of the trapezoidal rule (or equivalently Clenshaw–Curtis) as a function of the total number of evaluations $ N $ and the dimension $ d $? How would the convergence rates change (for both trapezoidal rule and Clenshaw–Curtis) if the function were smooth (analytic)?   

## Problem 3: Newton methods (10+10+10 points)

**(1)** The [Lambert W function](https://en.wikipedia.org/wiki/Lambert_W_function) is the root $w$ of the function $f(w) = w e^w - z$, which implicitly defines a function $w(z)$, starting with an initial guess $w = z$.   Use Newton's method to compute and plot $w(z)$ for a set of 100 points $z \in [0,1]$.   Check your answer against [`scipy.special.lambertw(z)`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.special.lambertw.html) in Python or [`LambertW.lambertw(z)`](https://github.com/robertdj/LambertW.jl) in Julia.

**(2)** Propose a Newton-like iteration $x_k \to x_{k+1}$ to find a root of any $f(x) = 0$ (not just Lambert W) that takes advantage of $f, f', f''$ at $x_k$ by making a *quadatic* approximation rather than a linear approximation.  (A quadratic approximation has two roots; which should you pick?  If you solve a quadratic equation $ax^2 + bx + c = 0$, you might want to compute the two roots as $\frac{-b - \sqrt{b^2 - 4ac}}{2a}$ and $\frac{2c}{-b - \sqrt{b^2 - 4ac}}$, assuming $b > 0$, to avoid disastrous cancellation errors for $|ac| \ll b^2$ as [discussed in class](https://github.com/mitmath/numerical_hub/blob/dd24729ecb3340b6551b1aaeb18afafdd5135fad/notes/Floating-Point-Intro.ipynb).)

**(3)** Apply your algorithm (2) to find a root of $\sin(x)$ starting from $x=3$, which *should* converge to $\pi$.   Give convincing numerical evidence that it is converging *cubically*: if $\epsilon_k$ is the error on step $k$, then the error on the next step is $O(\epsilon_k^3)$.  You may need to use arbitrary-precision arithmetic (via the `mpmath` package in Python or using `BigFloat` numbers in Julia) to see this clearly.